In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [66]:
skillType=pd.read_csv("vqa/vqa_skill_typ_train.csv",engine="python")

skillType.head(3)

,QID,IMG,QSN,ANS_TYP,TXT,OBJ,COL,CNT,OTH,ANS1,ANS2,ANS3,ANS4,ANS5,ANS6,ANS7,ANS8,ANS9,ANS10
0,100063007,COCO_train2014_000000100063.jpg,Are these trains moving?,yes/no,0,1,1,0,1,no,no,yes,no,yes,yes,no,no,no,yes
1,100140002,COCO_train2014_000000100140.jpg,Are the clouds visible?,yes/no,1,1,0,0,0,yes,no,no,no,yes,yes,yes,yes,no,no
2,100142000,COCO_train2014_000000100142.jpg,What number is on the shorts?,number,0,1,1,1,0,2,21,0,0,0,there is no number,10,no numbers shown,0,0


In [4]:
keyPhrase=pd.read_csv("vqa/vqa_train_keyphrases.csv", delimiter=";", engine="python",error_bad_lines=False)
keyPhrase.rename(columns={'oth,,,,,':'oth'}, inplace = True)
for i in range(0,len(keyPhrase)):
    keyPhrase.loc[i,'oth']=keyPhrase.loc[i,'oth'].replace(',', '')
keyPhrase.head(3)

Skipping line 47: ';' expected after '"'
Skipping line 108: ';' expected after '"'
Skipping line 186: ';' expected after '"'
Skipping line 211: ';' expected after '"'
Skipping line 238: ';' expected after '"'
Skipping line 309: ';' expected after '"'
Skipping line 338: ';' expected after '"'
Skipping line 479: ';' expected after '"'
Skipping line 551: ';' expected after '"'
Skipping line 569: ';' expected after '"'
Skipping line 813: ';' expected after '"'
Skipping line 856: ';' expected after '"'
Skipping line 1024: ';' expected after '"'
Skipping line 1035: ';' expected after '"'
Skipping line 1153: ';' expected after '"'
Skipping line 1183: ';' expected after '"'
Skipping line 1209: ';' expected after '"'
Skipping line 1223: ';' expected after '"'
Skipping line 1228: ';' expected after '"'
Skipping line 1250: ';' expected after '"'
Skipping line 1279: ';' expected after '"'
Skipping line 1286: ';' expected after '"'
Skipping line 1290: ';' expected after '"'
Skipping line 1370: ';' 

,qid,question,keyPhrases,obj,txt,col,cnt,oth
0,100063007,Are these trains moving?,['trains'],1,0,1,0,1
1,100140002,Are the clouds visible?,['clouds visible'],1,1,0,0,0
2,100142000,What number is on the shorts?,"['number', 'shorts']",1,0,1,1,0


In [33]:
#color recognition
colorRecognition=pd.read_csv("vqa/vqa_train_color_recognition.csv", delimiter=";", engine="python")
colorRecognition.head(3)
# The structure of VizWiz file

,qid,question,descriptions,tags,dominant_colors
0,100063007,Are these trains moving?,"[{'confidence': 0.9598418915543041, 'text': 'a...","['outdoor', 'track', 'train', 'transport', 'ca...","['White', 'Grey']"
1,100142000,What number is on the shorts?,"[{'confidence': 0.4043817227150979, 'text': 'a...","['person', 'outdoor', 'holding', 'young', 'boy...",['Grey']
2,101362004,Has the bun been toasted?,"[{'confidence': 0.9887643526628341, 'text': 'a...","['food', 'plate', 'table', 'fries', 'indoor', ...",['Brown']


In [38]:
print(colorRecognition.loc[0]["descriptions"])

[{'confidence': 0.9598418915543041, 'text': 'a train on a steel track'}]


In [67]:
colorRecognition['answer']=colorRecognition['descriptions']+colorRecognition['dominant_colors']

colorRecognition.head(3)

,qid,question,descriptions,tags,dominant_colors,answer
0,100063007,Are these trains moving?,"[{'confidence': 0.9598418915543041, 'text': 'a...","['outdoor', 'track', 'train', 'transport', 'ca...","['White', 'Grey']","[{'confidence': 0.9598418915543041, 'text': 'a..."
1,100142000,What number is on the shorts?,"[{'confidence': 0.4043817227150979, 'text': 'a...","['person', 'outdoor', 'holding', 'young', 'boy...",['Grey'],"[{'confidence': 0.4043817227150979, 'text': 'a..."
2,101362004,Has the bun been toasted?,"[{'confidence': 0.9887643526628341, 'text': 'a...","['food', 'plate', 'table', 'fries', 'indoor', ...",['Brown'],"[{'confidence': 0.9887643526628341, 'text': 'a..."


In [61]:
outputJson=output.to_json(orient='records')
print(outputJson)

[{"answer":"[{'confidence': 0.9598418915543041, 'text': 'a train on a steel track'}]['White', 'Grey']","question_id":100063007},{"answer":"[{'confidence': 0.4043817227150979, 'text': 'a young boy riding a skateboard'}]['Grey']","question_id":100142000},{"answer":"[{'confidence': 0.9887643526628341, 'text': 'a white plate topped with a sandwich and fries on a table'}]['Brown']","question_id":101362004}]


In [62]:
import json
with open("answers/color_des_domin.json", "w") as fp:
    fp.write(outputJson)


In [30]:
textRecognition=pd.read_csv("vqa/vqa_train_text_recognition.csv", delimiter=";", engine="python",
                       error_bad_lines=False)

textRecognition.rename(columns={'handwritten_text,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,':'handwritten_text'}, inplace = True)
for i in range(0,len(textRecognition)):
    textRecognition.loc[i,'handwritten_text']=textRecognition.loc[i,'handwritten_text'].replace(',,', '')
    textRecognition.loc[i,'handwritten_text']=textRecognition.loc[i,'handwritten_text'].replace('],', ']')
textRecognition.head(3)

Skipping line 52: ';' expected after '"'
Skipping line 66: ';' expected after '"'
Skipping line 75: ';' expected after '"'
Skipping line 86: ';' expected after '"'
Skipping line 87: ';' expected after '"'
Skipping line 90: ';' expected after '"'
Skipping line 122: ';' expected after '"'
Skipping line 141: ';' expected after '"'
Skipping line 241: ';' expected after '"'
Skipping line 286: ';' expected after '"'
Skipping line 311: ';' expected after '"'
Skipping line 352: ';' expected after '"'
Skipping line 137: Expected 4 fields in line 137, saw 5
Skipping line 167: Expected 4 fields in line 167, saw 5
Skipping line 179: Expected 4 fields in line 179, saw 5


,qid,question,ocr_text,handwritten_text
0,100140002,Are the clouds visible?,[],"['WAY', 'AHH HHHHHHHH TIKZ`` !', 'MOM', 'ANO',..."
1,10058001,A person with what kind of job would ride in t...,[],"['DOD`', 'THIET`', 'I', 'EEEEEE', 'WWW`. STATI..."
2,10125010,Was this picture taken in Texas?,[],"['PICKBILICA', 'LARRY`TE', '11 12', 'Com`ban`']"


In [31]:
print(textRecognition.loc[23]["handwritten_text"])

['On', 'SHINGA9\\', 'Enare', 'ZUG QUOT', 'rovote &', 'OHIOLAOUIHU!', 'MA ! OTMAP']


In [ ]:
#evaluate
contractions = {"aint": "ain't", "arent": "aren't", "cant": "can't", "couldve": "could've", "couldnt": "couldn't", \
							 "couldn'tve": "couldn't've", "couldnt've": "couldn't've", "didnt": "didn't", "doesnt": "doesn't", "dont": "don't", "hadnt": "hadn't", \
							 "hadnt've": "hadn't've", "hadn'tve": "hadn't've", "hasnt": "hasn't", "havent": "haven't", "hed": "he'd", "hed've": "he'd've", \
							 "he'dve": "he'd've", "hes": "he's", "howd": "how'd", "howll": "how'll", "hows": "how's", "Id've": "I'd've", "I'dve": "I'd've", \
							 "Im": "I'm", "Ive": "I've", "isnt": "isn't", "itd": "it'd", "itd've": "it'd've", "it'dve": "it'd've", "itll": "it'll", "let's": "let's", \
							 "maam": "ma'am", "mightnt": "mightn't", "mightnt've": "mightn't've", "mightn'tve": "mightn't've", "mightve": "might've", \
							 "mustnt": "mustn't", "mustve": "must've", "neednt": "needn't", "notve": "not've", "oclock": "o'clock", "oughtnt": "oughtn't", \
							 "ow's'at": "'ow's'at", "'ows'at": "'ow's'at", "'ow'sat": "'ow's'at", "shant": "shan't", "shed've": "she'd've", "she'dve": "she'd've", \
							 "she's": "she's", "shouldve": "should've", "shouldnt": "shouldn't", "shouldnt've": "shouldn't've", "shouldn'tve": "shouldn't've", \
							 "somebody'd": "somebodyd", "somebodyd've": "somebody'd've", "somebody'dve": "somebody'd've", "somebodyll": "somebody'll", \
							 "somebodys": "somebody's", "someoned": "someone'd", "someoned've": "someone'd've", "someone'dve": "someone'd've", \
							 "someonell": "someone'll", "someones": "someone's", "somethingd": "something'd", "somethingd've": "something'd've", \
							 "something'dve": "something'd've", "somethingll": "something'll", "thats": "that's", "thered": "there'd", "thered've": "there'd've", \
							 "there'dve": "there'd've", "therere": "there're", "theres": "there's", "theyd": "they'd", "theyd've": "they'd've", \
							 "they'dve": "they'd've", "theyll": "they'll", "theyre": "they're", "theyve": "they've", "twas": "'twas", "wasnt": "wasn't", \
							 "wed've": "we'd've", "we'dve": "we'd've", "weve": "we've", "werent": "weren't", "whatll": "what'll", "whatre": "what're", \
							 "whats": "what's", "whatve": "what've", "whens": "when's", "whered": "where'd", "wheres": "where's", "whereve": "where've", \
							 "whod": "who'd", "whod've": "who'd've", "who'dve": "who'd've", "wholl": "who'll", "whos": "who's", "whove": "who've", "whyll": "why'll", \
							 "whyre": "why're", "whys": "why's", "wont": "won't", "wouldve": "would've", "wouldnt": "wouldn't", "wouldnt've": "wouldn't've", \
							 "wouldn'tve": "wouldn't've", "yall": "y'all", "yall'll": "y'all'll", "y'allll": "y'all'll", "yall'd've": "y'all'd've", \
							 "y'alld've": "y'all'd've", "y'all'dve": "y'all'd've", "youd": "you'd", "youd've": "you'd've", "you'dve": "you'd've", \
							 "youll": "you'll", "youre": "you're", "youve": "you've"}
manualMap    = { 'none': '0',
							  'zero': '0',
							  'one': '1',
							  'two': '2',
							  'three': '3',
							  'four': '4',
							  'five': '5',
							  'six': '6',
							  'seven': '7',
							  'eight': '8',
							  'nine': '9',
							  'ten': '10'
							}
articles     = ['a',
							 'an',
							 'the'
							]
 

periodStrip  = re.compile("(?!<=\d)(\.)(?!\d)")
commaStrip   = re.compile("(\d)(\,)(\d)")
punct        = [';', r"/", '[', ']', '"', '{', '}',
				'(', ')', '=', '+', '\\', '_', '-',
				'>', '<', '@', '`', ',', '?', '!']



resAns      = res[quesId]['answer']
			resAns      = resAns.replace('\n', ' ')
			resAns      = resAns.replace('\t', ' ')
			resAns      = resAns.strip()
			resAns      = self.processPunctuation(resAns)
			resAns      = self.processDigitArticle(resAns)
gtAcc  = []
			gtAnswers = [ans['answer'] for ans in gts[quesId]['answers']]
			if len(set(gtAnswers)) > 1: 
				for ansDic in gts[quesId]['answers']:
					ansDic['answer'] = self.processPunctuation(ansDic['answer'])
			for gtAnsDatum in gts[quesId]['answers']:
				otherGTAns = [item for item in gts[quesId]['answers'] if item!=gtAnsDatum]
				matchingAns = [item for item in otherGTAns if item['answer'] in resAns]
				acc = min(1, float(len(matchingAns))/3)
				gtAcc.append(acc)
			quesType    = gts[quesId]['question_type']
			ansType     = gts[quesId]['answer_type']
			avgGTAcc = float(sum(gtAcc))/len(gtAcc)
			accQA.append(avgGTAcc)
			if quesType not in accQuesType:
				accQuesType[quesType] = []
			accQuesType[quesType].append(avgGTAcc)
			if ansType not in accAnsType:
				accAnsType[ansType] = []
			accAnsType[ansType].append(avgGTAcc)
			self.setEvalQA(quesId, avgGTAcc)
			self.setEvalQuesType(quesId, quesType, avgGTAcc)
			self.setEvalAnsType(quesId, ansType, avgGTAcc)
			if step%100 == 0:
				self.updateProgress(step/float(len(quesIds)))
			step = step + 1

		self.setAccuracy(accQA, accQuesType, accAnsType)
		print("Done computing accuracy")
	
	def processPunctuation(inText):
		outText = inText
		for p in punct:
			if (p + ' ' in inText or ' ' + p in inText) or (re.search(commaStrip, inText) != None):
				outText = outText.replace(p, '')
			else:
				outText = outText.replace(p, ' ')	
		outText = periodStrip.sub("",
									 outText,
									 re.UNICODE)
		return outText
	
	def processDigitArticle(self, inText):
		outText = []
		tempText = inText.lower().split()
		for word in tempText:
			word = self.manualMap.setdefault(word, word)
			if word not in self.articles:
				outText.append(word)
			else:
				pass
		for wordId, word in enumerate(outText):
			if word in self.contractions: 
				outText[wordId] = self.contractions[word]
		outText = ' '.join(outText)
		return outText